In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
train_file = "/content/drive/My Drive/Colab Notebooks/dataset_kulit_skripsi.zip"

print(train_file)

In [ ]:
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
with zipfile.ZipFile(train_file, 'r') as z:
    z.extractall()

In [ ]:
def load_image(file_path):
    return plt.imread(file_path)

def extract_label(file_name):
    if "domba" in file_name:
        return 0
    elif  "imitasi" in file_name:
        return 1
    elif "kambing" in file_name:
        return 2
    else:
        return 3

train_path = "./dataset_kulit_skripsi/"
image_files = os.listdir(train_path)

class_name = ['Domba', 'Imitasi', 'Kambing', 'Sapi']

In [ ]:
train_images_x = [load_image(train_path + file) for file in image_files]

In [ ]:
train_labels = [extract_label(file) for file in image_files]

print(type(train_images_x))
print(type(train_labels))
print('train_labels =', train_labels)
print(len(train_labels))

In [ ]:
# plt.imshow(train_images[1])

# print('Gambar', train_labels[1], '=', class_name[train_labels[1]])
# print('Shape =', train_images[1].shape)

In [ ]:
def preprocess_grayscale(img, side=448):
    min_side = min(img.shape[0], img.shape[1])
    img = img[:min_side, :min_side]
    img = cv2.resize(img, (side, side))
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    return img_gray / 255.0

def preprocess_adaptive_gaussian(gambar, side=448):
    min_side = min(gambar.shape[0], gambar.shape[1])
    gambar = gambar[:min_side, :min_side]
    gambar = cv2.resize(gambar, (side, side))
    gambar = cv2.cvtColor(gambar, cv2.COLOR_BGR2GRAY)

    img_gaussian = cv2.adaptiveThreshold(gambar, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)

    return img_gaussian / 255.0

# def preprocess_otsu_threshold(gambar2, side=384):
#     min_side = min(gambar2.shape[0], gambar2.shape[1])
#     gambar2 = gambar2[:min_side, :min_side]
#     gambar2 = cv2.resize(gambar2, (side, side))
#     gambar2 = cv2.cvtColor(gambar2, cv2.COLOR_BGR2GRAY)

#     _, citra_otsu = cv2.threshold(gambar2, 125, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#     return citra_otsu / 255.0

In [ ]:
# var = (preprocess_grayscale(train_images_x[0]))
# display(pd.DataFrame(var))

In [ ]:
print(type(preprocess_adaptive_gaussian(train_images_x[21])))
print(type(train_images_x[21]))
print(type(train_images_x))
print(len(train_images_x))

In [ ]:
for x in range(len(train_images_x)):
  train_images_x[x] = preprocess_grayscale(train_images_x[x])

print(len(train_images_x))

In [ ]:
train_images_y = [load_image(train_path + file) for file in image_files]

In [ ]:
for x in range(len(train_images_y)):
  train_images_y[x] = preprocess_adaptive_gaussian(train_images_y[x])

print(len(train_images_y))

In [ ]:
print(type(train_images_x))
print(type(train_images_y))

temp_train_images = list()
temp_train_labels = list()

temp_train_images.extend(train_images_x)
temp_train_images.extend(train_images_y)

temp_train_labels.extend(train_labels)
temp_train_labels.extend(train_labels)

print(type(temp_train_images))
print(len(temp_train_images))
print(len(temp_train_labels))

In [ ]:
temp_train_images = np.array(temp_train_images)

temp_train_images.shape

In [ ]:
%matplotlib inline

plt.figure(figsize=(18, 6))

plt.subplot(1,2,1)
plt.title("Citra Grayscale")
plt.imshow(temp_train_images[8], cmap="gray")

plt.subplot(1,2,2)
plt.title("Citra Gaussian")
plt.imshow(temp_train_images[648], cmap="gray")

print('Gambar', temp_train_labels[8], '=', class_name[temp_train_labels[8]])
print('Shape =', temp_train_images[8].shape)

print('\nGambar', temp_train_labels[648], '=', class_name[temp_train_labels[648]])
print('Shape =', temp_train_images[648].shape)

In [ ]:
print(temp_train_images[0].shape)

print(len(temp_train_images))

In [ ]:
temp_train_images = np.expand_dims(temp_train_images, axis=-1)
temp_train_labels = np.array(temp_train_labels)

print(temp_train_images.shape, 'train_images')
print(temp_train_labels.shape, 'train_labels')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(temp_train_images, temp_train_labels, test_size=0.2, random_state=42)

In [ ]:
print(temp_train_images.shape, 'train_images')
print(temp_train_labels.shape, 'train_labels')
print('---------------------------------')

temp_train_images = X_train
temp_train_labels = y_train
val_images = X_test
val_labels = y_test

# test_images = X_test
# test_labels = y_test

print(temp_train_images.shape[0], 'train_images')
print(val_images.shape[0], 'validation data\n')

print(temp_train_labels.shape[0], 'train_labels')
print(val_labels.shape[0], 'validation labels')

In [ ]:
print(temp_train_images.shape)
print(temp_train_labels.shape)
print(val_images.shape)

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

df = pd.DataFrame(val_labels)

df.columns=["Nilai"]

for index, row in df.iterrows():
  if row['Nilai'] == 0:
    df.loc[index, 'Jenis'] = 'Domba'
  elif row ['Nilai'] == 1:
    df.loc[index, 'Jenis'] = 'Imitasi'
  elif row ['Nilai'] == 2:
    df.loc[index, 'Jenis'] = 'Kambing'
  else:
    df.loc[index, 'Jenis'] = "Sapi"
        
df[['Nilai','Jenis']]
df.to_csv('test_data.csv')
print(df)

In [ ]:
# plt.imshow(val_images[99])

print('Gambar', val_labels[0], '=', class_name[val_labels[0]])

In [ ]:
import keras
import tensorflow as tf

import tensorboard
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

print("Tensorflow:", tf.__version__, "\n")
print("Keras:", keras.__version__, "\n")
print("OpenCV:", cv2.__version__, "\n")
print("Tensorboard:", tensorboard.__version__)

In [ ]:
temp_train_images.shape[1:]

In [ ]:
# # Clear any logs from tensoboard previous runs
# !rm -rf ./logs/ 

# layers = [
#     tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1,1), padding="same", activation=tf.keras.activations.relu, input_shape = temp_train_images.shape[1:]),
#     tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

#     tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
#     tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

#     tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
#     tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

#     tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
#     tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

#     tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
#     tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

#     tf.keras.layers.Conv2D(filters=160, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
#     tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

#     # tf.keras.layers.Dropout(0.2),

#     tf.keras.layers.Flatten(),

#     tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu),

#     tf.keras.layers.Dense(units=4, activation=tf.keras.activations.softmax)
# ]

In [ ]:
# Clear any logs from tensoboard previous runs
!rm -rf ./logs/ 

layers = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu, input_shape = temp_train_images.shape[1:]),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=160, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=192, kernel_size=(3, 3), strides=(1,1), padding="same", activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=128, activation=tf.keras.activations.relu),

    tf.keras.layers.Dropout(rate=0.2),

    tf.keras.layers.Dense(units=4, activation=tf.keras.activations.softmax)
]

In [ ]:
model = tf.keras.Sequential(layers)

print(model.summary())

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model-CNN.png', show_shapes=True, show_layer_names=True,
    rankdir='TB', expand_nested=False, dpi=72
)

import IPython
from IPython.display import Image
Image('model-CNN.png')

In [ ]:
import sklearn.metrics
import itertools
from packaging import version
from six.moves import range
import io

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def plot_confusion_matrix(cm, class_name):
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_name))
  plt.xticks(tick_marks, class_name, rotation=45)
  plt.yticks(tick_marks, class_name)

  # Normalize the confusion matrix.
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

# Clear out prior logging data.
!rm -rf logs/image

logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(val_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_name=class_name)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('sparse_categorical_accuracy')>0.90 and logs.get('val_sparse_categorical_accuracy')>0.90):
      print("\n\n")
      print("Akurasi telah mencapai nilai yang ditentukan!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
num_epoch = 300
num_batch = 64

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), # menghitung crossentropy loss antara label dengan prediksi
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()], # menghitung seberapa sering prediksi cocok dengan label integer
    )

# print(temp_train_images.shape, 'train_images')
# print(temp_train_labels.shape, 'train_labels\n')
# print(val_images.shape, 'val_images')
# print(val_labels.shape, 'val_labels\n')

# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

mod = model.fit(
    temp_train_images,
    temp_train_labels,
    epochs = num_epoch,
    batch_size = num_batch,
    verbose = 2,
    callbacks = [tensorboard_callback, cm_callback, callbacks],
    validation_data = (val_images, val_labels)
    )

# model.save_weights("skripsi_cnn.h5")

model.save("model_skripsi.h5")

# %tensorboard --logdir logs/fit

In [ ]:
# Start TensorBoard.
%tensorboard --logdir logs/image

In [ ]:
print(mod.history.keys())

plt.figure(figsize=(16, 5))

plt.subplot(121)
plt.title('Nilai Akurasi')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.plot(mod.history['sparse_categorical_accuracy'], 'tab:green')
plt.plot(mod.history['val_sparse_categorical_accuracy'], 'tab:orange')
plt.legend(['sparse_categorical_accuracy', 'validation_accuracy'], loc='upper left')
plt.grid(b=True, which='major', color='#D1D1D1', linestyle='-')
# plt.tight_layout()
# plt.savefig("acc.png")

plt.subplot(122)
plt.title('Nilai Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend([ 'loss'], loc='upper right')
plt.plot(mod.history['loss'], 'tab:green')
plt.plot(mod.history['val_loss'], 'tab:orange')
plt.legend(['loss', 'validation_loss'], loc='upper left')
plt.grid(b=True, which='major', color='#D1D1D1', linestyle='-')
# plt.tight_layout()

plt.savefig("data-training.png")

plt.suptitle('Hasil Penilaian Akurasi dan Loss')
plt.show()

In [ ]:
plt.title('Accuracy')
# plt.figure(figsize=(18, 6))
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.plot(mod.history['sparse_categorical_accuracy'], 'tab:green')
plt.plot(mod.history['val_sparse_categorical_accuracy'], 'tab:orange')
plt.legend(['sparse_categorical_accuracy', 'validation_accuracy'], loc='upper left')
plt.grid(b=True, which='major', color='#D1D1D1', linestyle='-')
plt.tight_layout()
plt.savefig("acc.png")

In [ ]:
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend([ 'loss'], loc='upper right')
plt.plot(mod.history['loss'], 'tab:green')
plt.plot(mod.history['val_loss'], 'tab:orange')
plt.legend(['loss', 'validation_loss'], loc='upper left')
plt.grid(b=True, which='major', color='#D1D1D1', linestyle='-')
plt.tight_layout()
plt.savefig("loss.png")

In [ ]:
# print(np.argmax(model.predict(val_images)))
# print('Image Class :', model.predict_classes(val_images))
# print(len(model.predict_classes(val_images)))

In [ ]:
score = model.evaluate(val_images, val_labels, verbose=2)

print('Test loss =', score[0])
print('Test accuracy =', score[1])

In [ ]:
# from sklearn.metrics import classification_report

# print (classification_report(val_labels, test_pred))

In [ ]:
from google.colab import files
unggah_data_test = files.upload()

In [ ]:
eval_images = [preprocess_adaptive_gaussian(load_image(file)) for file in unggah_data_test.keys()]
eval_model = tf.keras.Sequential(layers)

# eval_model.load_weights("model_kulit_skripsi.h5")
eval_model.load_weights("model_skripsi.h5")

eval_predictions = eval_model.predict(np.expand_dims(eval_images, axis=-1))
print(type(eval_predictions))

cols = 6
rows = np.ceil(len(eval_images) / cols)
fig = plt.gcf()
fig.set_size_inches(cols * 6, rows * 6)

for i in range(len(eval_images)):
    
    print(np.argmax(eval_predictions[i]))
    print(eval_predictions[i])

    plt.subplot(rows, cols, i+1)
    plt.imshow(eval_images[i], cmap="gray")
    
    judul = ''
    if (np.argmax(eval_predictions[i]) == 0) :
        kelas = 'Sheep'
    elif (np.argmax(eval_predictions[i]) == 1):
        kelas = 'Synthetic'
    elif (np.argmax(eval_predictions[i]) == 2):
        kelas = 'Goat'
    else:
        kelas = 'Cow'
    
    plt.title(kelas)
    plt.axis('on')